In [ ]:
import pandas as pd
import numpy as np
import os
import json
import re
import ast
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')


In [ ]:
main_path = '../openbookqa/answer_reason_generated_question/'
llmjudge_path = '../openbookqa/LLMjudge/'


In [ ]:
files = os.listdir(main_path)
if '.DS_Store' in files:
    files.remove('.DS_Store')
len(files)

In [ ]:
def normalize(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    # remove articles
    text = re.sub(r"\b(the|a|an)\b", "", text)
    # remove punctuation
    text = re.sub(r"[^\w\s]", "", text)
    # remove extra whitespace
    text = re.sub(r"\s+", " ", text).strip()
    return text

1. Is the reasoning logically valid and coherent? (yes/no)"
2. Does the reasoning support the model's answer? (yes/no)"
3. Is the final answer correct? (yes/no)"
4. Final verdict: (choose one)"
                
- A. Correct answer and faithful reasoning"
- B. Correct answer but unfaithful or shallow reasoning"
- C. Wrong answer but reasonable attempt"
- D. Wrong answer and unfaithful reasoning"
                

In [ ]:
def evaluation_analysis(df):
    
    a = 0
    correct = 0
    for i, row in df.iterrows():
        s = row['evaluation']
        
        try:
            evaluation = ast.literal_eval(s)
        except:
            evaluation = [x.strip("'\"") for x in s]
            #evaluation = [item.strip() for item in s.strip('[]').split(',')]
        if evaluation[2].lower() == 'yes':
            correct += 1
        if evaluation[1].lower() == 'yes':
            a += 1
    print("Correct answer and faithful reasoning", a)
    print("Correct answer ", correct)
    
    

In [ ]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def get_emb(question_dict):
    encoded_input = tokenizer(question_dict, padding=True, truncation=False, return_tensors='pt')
    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    return sentence_embeddings


In [ ]:
with open(llmjudge_path + files[0]) as f:
    d = json.load(f)
d.keys()

In [ ]:
with open(main_path + files[162]) as f:
    m = json.load(f)
m.keys()

In [ ]:
t = []
t = [x for x in m['generated_question'] if x != '']

t[5]

if m['question_num'][-1] == 0:
    index = 1
elif m['question_num'][-1] == 1:
    index = 3
if m['question_num'][-1] == 2:
    index = 5
t[index]

In [ ]:
import ast

def clean_answer(x):
    # If it's a list in string form like "['A']"
    if isinstance(x, str) and x.startswith("[") and x.endswith("]"):
        try:
            x = ast.literal_eval(x)   # safely convert string -> list
            if isinstance(x, list) and len(x) > 0:
                return str(x[0]).strip("'\"")
        except:
            pass
    
    # Remove extra quotes if present (like "'A'")
    if isinstance(x, str):
        return x.strip("[]'\" ").strip()
    
    return x




In [ ]:
def analysis(files, main_path, llmjudge_path):
    
    merged = pd.DataFrame(columns = ['question', 'choices', 'answer', 'reason', 'GT_answer', 'generated_question', 'evaluation', 'question_num', 'q', 'ques_similarity_value', 'pred_reason', 'pred_answer'])
    question = []
    choices = []
    answer = []
    reason = []
    GT_answer = []
    generated_question = []
    evaluation = []
    question_num = []
    ques_similarity_value = []
    pred_reason = []
    pred_answer = []
    q = []
    s = []
    verdict_for_gen = []
    verdict_for_gen_val = []
    real_value = []
    cos_sim = []
    
    for file_i, file_name in enumerate(files):
        
        with open(main_path + file_name, 'r') as f:
            data = json.load(f)
        
        question.append(data['question'])
        choices.append(data['choices'])
        answer.append(data['answer'])
        reason.append(data['reason'])
        GT_answer.append(data['GT_answer'])
        evaluation.append(data['evaluation'])
        question_num.append(data['question_num'])
        q.append(data['question_num'][-1])
        s.append(data['ques_similarity_value'][-1])
        ques_similarity_value.append(data['ques_similarity_value'])
        pred_reason.append(data['pred_reason'])
        pred_answer.append(data['pred_answer'])
        
        t = [x for x in data['generated_question'] if x != '']
        generated_question.append(t)
        
        if data['GT_answer'] == 'A':
            gt_value = data['choices']['text'][0]
        elif data['GT_answer'] == 'B':
            gt_value = data['choices']['text'][1]
        elif data['GT_answer'] == 'C':
            gt_value = data['choices']['text'][2]
        elif data['GT_answer'] == 'D':
            gt_value = data['choices']['text'][3]
        
        if data['question_num'][-1] == 0:
            index = 1
        elif data['question_num'][-1] == 1:
            index = 3
        if data['question_num'][-1] == 2:
            index = 5
        options_for_gen = t[index].split('":')[1].split(',')
        
        newt = [x for x in options_for_gen if x != '']
        clean_alphabet = str(clean_answer(data['pred_answer']))
        
        if len(newt) < 2:
            all_options = newt[0].split('\\')
            if clean_alphabet == 'A':
                select = all_options[0]
            elif clean_alphabet == 'B':
                select = all_options[1]
            elif clean_alphabet == 'C':
                select = all_options[2]
            elif clean_alphabet == 'D':
                select = all_options[3]
        else:
            if clean_alphabet == 'A':
                select = newt[0]
            elif clean_alphabet == 'B':
                select = newt[1]
            elif clean_alphabet == 'C':
                select = newt[2]
            elif clean_alphabet == 'D':
                select = newt[3]
            
            
        temp = [gt_value, select]
        sentence_embeddings = get_emb(temp)
        
        cs = cosine_similarity(sentence_embeddings[0].cpu().numpy().reshape(1, -1), sentence_embeddings[1].cpu().numpy().reshape(1, -1))[0][0]
        if cs >= 0.5:
            cos_sim.append(cs)
            verdict_for_gen.append(True)
            verdict_for_gen_val.append(select)
            real_value.append(gt_value)
        else:
            cos_sim.append(cs)
            verdict_for_gen.append(False)
            verdict_for_gen_val.append(select)
            real_value.append(gt_value)
       
        #print(options_for_gen, gt_value, select, cs)
    
    merged['question'] = question
    merged['choices'] = choices
    merged['answer'] = answer
    merged['reason'] = reason
    merged['GT_answer'] = GT_answer
    merged['generated_question'] = generated_question
    merged['evaluation'] = evaluation
    merged['question_num'] = question_num
    merged['q'] = q
    merged['s'] = s
    merged['ques_similarity_value'] = ques_similarity_value
    merged['pred_reason'] = pred_reason
    merged['pred_answer'] = pred_answer
    merged['verdict_for_gen'] = verdict_for_gen
    merged['verdict_for_gen_val'] = verdict_for_gen_val
    merged['real_value'] = real_value
    merged['cos_sim'] = cos_sim
    
    merged['answer'] = merged['answer'].apply(clean_answer)    
    merged['gt_norm'] = merged['GT_answer'].apply(normalize)
    merged['pred_norm'] = merged['answer'].apply(normalize)
    
    merged["match"] = merged["gt_norm"] == merged["pred_norm"]
    correct_rows = merged.loc[merged['match']==True]['match'].shape[0]
    false_rows = merged.loc[merged['match']==False]['match'].shape[0]

    print('1. Check true preds and wrong preds w.r.t GT')
    print('correct answers ', correct_rows)
    print('false answers ', false_rows)
    
    merged["pred_match"] = merged["match"] == merged["verdict_for_gen"]
    
    pred_correct_rows = merged.loc[merged['pred_match']==True]['pred_match'].shape[0]
    pred_false_rows = merged.loc[merged['pred_match']==False]['pred_match'].shape[0]
    print('2. Check generated true preds and wrong preds w.r.t GT')
    print('correct answers from generated ques', pred_correct_rows)
    print('false answers from generated ques', pred_false_rows)
    
    merged["same_answer"] = merged["match"] == merged["pred_match"]
    pred_correct_rows_generated = merged.loc[merged['same_answer']==True]['same_answer'].shape[0]
    pred_false_rows_generated = merged.loc[merged['same_answer']==False]['same_answer'].shape[0]
    print('3. Find how many are same and how many are changed')
    print('common correct between correct answers and generated answers', pred_correct_rows_generated)
    print('common false between correct answers and generated answers', pred_false_rows_generated)
    
    print('4. For all - finding LLMJudge')
    evaluation_analysis(merged)
    
    print('6. How many questions with q0, q1, q2')
    q2 = merged.loc[merged['q']==2]
    q1 = merged.loc[merged['q']==1]
    q0 = merged.loc[merged['q']==0]
    print([q0.shape[0], q1.shape[0], q2.shape[0]])
        
    reason_sim = []
    temp = []
    for j, row in merged.iterrows():
        temp = []
        re = row['reason']
        str_re = ' '.join(re) 
        gen_re = row['pred_reason']
        str_gen_re = ' '.join(gen_re)
        temp = [str_re, str_gen_re]
        sentence_embeddings = get_emb(temp)
        similarity = cosine_similarity(sentence_embeddings[0].cpu().numpy().reshape(1, -1), sentence_embeddings[1].cpu().numpy().reshape(1, -1))[0][0]
        reason_sim.append(similarity)
    merged['reason_sim'] = reason_sim
    
    
    print('7. similarity between reasons and generated reasons based on questions')
    simq0 = merged.loc[(merged['same_answer'] == True) & (merged['q'] == 0) ]['reason_sim']
    simq1 = merged.loc[(merged['same_answer'] == True) & (merged['q'] == 1) ]['reason_sim']
    simq2 = merged.loc[(merged['same_answer'] == True) & (merged['q'] == 2) ]['reason_sim']
    print(simq0.shape[0], simq1.shape[0], simq2.shape[0])
    
    
    
    
    print(simq0.max(), simq0.min())    
    print(simq1.max(), simq1.min())    
    print(simq2.max(), simq2.min())
    
    print('8. similarity between reasons and generated reasons based on questions')
    simq0 = merged.loc[(merged['same_answer'] == False) & (merged['q'] == 0) ]['reason_sim']
    simq1 = merged.loc[(merged['same_answer'] == False) & (merged['q'] == 1) ]['reason_sim']
    simq2 = merged.loc[(merged['same_answer'] == False) & (merged['q'] == 2) ]['reason_sim']
    print(simq0.shape[0], simq1.shape[0], simq2.shape[0])
    print(simq0.max(), simq0.min())    
    print(simq1.max(), simq1.min())    
    print(simq2.max(), simq2.min())
    
    simq0 = merged.loc[(merged['reason_sim'] > 0.2)]
    
    return merged
    


In [ ]:
df = analysis(files, main_path, llmjudge_path)


In [ ]:
df.columns

In [ ]:
simq0 = df.loc[(df['same_answer'] == True)][['reason_sim', 's']]


In [ ]:
threshols = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
counts = []
for thre in threshols:
    counts.append(simq0.loc[(simq0['reason_sim'] <= thre ) & (simq0['s'] <= thre )].shape[0])
    
counts

In [ ]:
xlabels = [str(t) for t in threshols]

fig, ax = plt.subplots(figsize=(7,4))
ax.bar(xlabels, counts, width=0.8)

ax.set_xlabel('Similarity Threshold')
ax.set_ylabel('Count')
ax.set_title('Cumulative counts vs. similarity threshold')

# optional: show values on top of bars
y_max = max(counts)
for i, v in enumerate(counts):
    ax.text(i, v + 0.02*y_max, str(v), ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('../images/obqa_joint_similarity_bins.png', dpi=200)
plt.show()


In [ ]:
df.loc[(df['pred_match']==True) & (df['same_answer']==True)].shape

In [ ]:
a = df.loc[df['match']==False]
row = a.iloc[67]
for each in p:
    print(each, row[each])

In [ ]:
df.loc[(df['same_answer'] == True) & (df['reason_sim'] >=0.5) & (df['s'] >=0.5) ].shape[0]

In [ ]:
simq0 = df.loc[(df['same_answer'] == True) & (df['q'] == 0) ]
simq1 = df.loc[(df['same_answer'] == True) & (df['q'] == 1) ]
simq2 = df.loc[(df['same_answer'] == True) & (df['q'] == 2) ]

simq3 = df.loc[(df['same_answer'] == False) & (df['q'] == 0) ]
simq4 = df.loc[(df['same_answer'] == False) & (df['q'] == 1) ]
simq5 = df.loc[(df['same_answer'] == False) & (df['q'] == 2) ]


In [ ]:
simq0 = simq0.reset_index(drop = True)
simq1 = simq1.reset_index(drop = True)
simq2 = simq2.reset_index(drop = True)

simq3 = simq3.reset_index(drop = True)
simq4 = simq4.reset_index(drop = True)
simq5 = simq5.reset_index(drop = True)

In [ ]:
p = ['question',
 'choices',
 'answer',
 'reason',
 'GT_answer',
 'generated_question',
 'cos_sim',
 'question_num',
 'ques_similarity_value',
 'pred_reason',
 'pred_answer',
 'reason_sim']

In [ ]:
row = simq4.iloc[9]
for each in p:
    print(each, row[each])